In [18]:
import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile
import cv2

from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as vis_util

# Definizione stream video
cap = cv2.VideoCapture(0)  # Cambia il valore solo se hai più webcam

MODEL_NAME = 'targa_graph'
DOWNLOAD_BASE = 'http://download.tensorflow.org/models/object_detection/'

# Path di frozen detection graph del modello attualmente utilizzato
PATH_TO_CKPT = MODEL_NAME + '/frozen_inference_graph.pb'

# Lista delle stringhe usate per i label
PATH_TO_LABELS = os.path.join('training', 'object-detection.pbtxt')

# Numero di classi da identificare
NUM_CLASSES = 1
  
# This is needed since the notebook is stored in the object_detection folder. 
sys.path.append("..") 
  
# Import utilites 
from utils import label_map_util 
from utils import visualization_utils as vis_util 
  
IMAGE_NAME = 'traffico.jpg'  # Path dell'immagine
  
# Grab path to current working directory 
CWD_PATH = '/home/andrea/.conda/envs/tensorflow/models/research/object_detection/test_images/'

  
  
# Path to image 
PATH_TO_IMAGE = os.path.join(CWD_PATH, IMAGE_NAME) 

  
# Load the label map. 
label_map = label_map_util.load_labelmap(PATH_TO_LABELS) 
categories = label_map_util.convert_label_map_to_categories( 
        label_map, max_num_classes = NUM_CLASSES, use_display_name = True) 
category_index = label_map_util.create_category_index(categories) 
  
# Load the Tensorflow model into memory. 
detection_graph = tf.Graph() 
with detection_graph.as_default(): 
    od_graph_def = tf.GraphDef() 
    with tf.gfile.GFile(PATH_TO_CKPT, 'rb') as fid: 
        serialized_graph = fid.read() 
        od_graph_def.ParseFromString(serialized_graph) 
        tf.import_graph_def(od_graph_def, name ='') 
  
    sess = tf.Session(graph = detection_graph) 
  
# Define input and output tensors (i.e. data) for the object detection classifier 
  
# Input tensor is the image 
image_tensor = detection_graph.get_tensor_by_name('image_tensor:0') 
  
# Output tensors are the detection boxes, scores, and classes 
# Each box represents a part of the image where a particular object was detected 
detection_boxes = detection_graph.get_tensor_by_name('detection_boxes:0') 
  
# Each score represents level of confidence for each of the objects. 
# The score is shown on the result image, together with the class label. 
detection_scores = detection_graph.get_tensor_by_name('detection_scores:0') 
detection_classes = detection_graph.get_tensor_by_name('detection_classes:0') 
  
# Number of objects detected 
num_detections = detection_graph.get_tensor_by_name('num_detections:0') 
  
# Load image using OpenCV and 
# expand image dimensions to have shape: [1, None, None, 3] 
# i.e. a single-column array, where each item in the column has the pixel RGB value 
image = cv2.imread(PATH_TO_IMAGE) 
image_expanded = np.expand_dims(image, axis = 0) 
  
# Perform the actual detection by running the model with the image as input 
(boxes, scores, classes, num) = sess.run( 
    [detection_boxes, detection_scores, detection_classes, num_detections], 
    feed_dict ={image_tensor: image_expanded}) 
  
# Draw the results of the detection (aka 'visualize the results') 
  
vis_util.visualize_boxes_and_labels_on_image_array( 
    image, 
    np.squeeze(boxes), 
    np.squeeze(classes).astype(np.int32), 
    np.squeeze(scores), 
    category_index, 
    use_normalized_coordinates = True, 
    line_thickness = 8, 
    min_score_thresh = 0.60) 
  
# All the results have been drawn on the image. Now display the image. 
cv2.imshow('Object detector', image) 
  
# Press any key to close the image 
cv2.waitKey(0)
# Clean up 
cv2.destroyAllWindows() 